In [ ]:
!pip install -q kaggle keras-preprocessing
!nvidia-smi -L

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.6 MB/s eta 0:00:00
GPU 0: Tesla T4 (UUID: GPU-b05279dd-124a-ad13-cb09-400605a96725)


In [ ]:
import tensorflow as tf
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.image as mat_img
import matplotlib.pyplot as plt
import os
from kaggle.api.kaggle_api_extended import KaggleApi


In [ ]:
os.environ['KAGGLE_USERNAME'] = "fellahsamy"
os.environ['KAGGLE_KEY'] = "236450079c9371bf4bac63b12dd62174"


def download_and_extract_kaggle_dataset(dataset_name, destination, unzip=True, quiet=False):
    # Set up Kaggle API credentials
    api = KaggleApi()
    api.authenticate()

    # Download the Kaggle dataset to the specified destination folder
    api.dataset_download_files(dataset_name, path=destination, unzip=unzip, quiet=quiet)


download_and_extract_kaggle_dataset('paramaggarwal/fashion-product-images-small', 'data/', unzip=True, )

100%|██████████| 565M/565M [00:05<00:00, 100MB/s]


In [ ]:


class FashionModel(tf.keras.Model):
    def __init__(self, input_shape, output_shape):
        super().__init__()
        self.features_layer = tf.keras.applications.ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape, pooling='avg')
        self.n_1_layer = tf.keras.layers.Dense(1024)
        self.output_layer = tf.keras.layers.Dense(output_shape, activation ='sigmoid')
        self.features_layer.trainable = False
        
    
    def call(self, inputs):
        features = self.features_layer(inputs)
        n_1_features = self.n_1_layer(features)
        return self.output_layer(n_1_features)
        
def read_img(row):
    
    try:
        path_imgs ="./data/images"
        file_name = str(row['id'])+'.jpg'
        file_name= os.path.join(path_imgs,file_name)
        image = mat_img.imread(file_name)
        row['image_shape'] = image.shape
        row['image']=image
    except FileNotFoundError as e:
        row['image_shape'] = None
        row['image']= None
    return row

def convert_single_chanel(df):
    for index,image_info in df.iterrows():
        if image_info.image.ndim == 2:
            new_image = np.zeros(shape=(*image_info.image.shape,3),dtype=np.int_)
            new_image[:,:,0]=image_info.image
            new_image[:,:,1]=image_info.image
            new_image[:,:,2]=image_info.image
            image_info['image'] = new_image
            image_info['image_shape'] = new_image.shape
            df.loc[index]=image_info
        
    return df

def resize_images(df):
    for index,image_info in df.iterrows():
        if image_info.image.shape != (80, 60, 3):
            img_resized = tf.image.resize(image_info['image'],[80,60]).numpy().astype(np.int_)
            image_info['image'] = img_resized
            image_info['image_shape'] = img_resized.shape
            df.loc[index]=image_info
        
    return df



if __name__=='__main__':
    categories_columns = ['gender','masterCategory','subCategory','articleType','baseColour','season']
    meta_info = "./data/styles.csv"
    print('Loading meta data...')
    all_data =pd.read_csv(meta_info,on_bad_lines='skip') 
    all_data = pd.get_dummies(all_data, columns=categories_columns, prefix='cat_')
    
    # total number of labels ('gender','masterCategory',...) = 286 label
    fashion_model = FashionModel(input_shape = (80,60,3),output_shape = 286)
    fashion_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["binary_accuracy",tf.keras.metrics.Recall(),tf.keras.metrics.Precision()])
    
    
    # charger les images en 10 fois pour eviter de surcharger la RAM
    for sub_dataset in np.array_split(all_data, 10):
        print('Loading images...')
        data = sub_dataset.apply(read_img , axis=1)
        data = data.dropna(subset=['image'])
        print('Preprocessing images...')
        data = convert_single_chanel(data)
        data = resize_images(data)
        y = data[[column for column in data.columns if column[0:4]=='cat_']]
        x = data.image.to_list()
        x = np.array(x)
        print('Preprocessing TF...')
        x = tf.keras.applications.resnet.preprocess_input(x) 
        print(f'Fitting model with {y.shape[0]} lines...')
        fashion_model.fit(x=x, y=y, batch_size=48, epochs=2)


Loading meta data...
94765736/94765736 [==============================] - 0s 0us/step
Loading images...
Preprocessing images...
Preprocessing TF...
Fitting model with 4443 lines...
Epoch 1/2
93/93 [==============================] - 15s 30ms/step - loss: 0.0743 - binary_accuracy: 0.9788 - recall: 0.6008 - precision: 0.5348
Epoch 2/2
93/93 [==============================] - 2s 23ms/step - loss: 0.0348 - binary_accuracy: 0.9894 - recall: 0.7200 - precision: 0.7981
Loading images...
Preprocessing images...
Preprocessing TF...
Fitting model with 4442 lines...
Epoch 1/2
93/93 [==============================] - 3s 29ms/step - loss: 0.0431 - binary_accuracy: 0.9878 - recall: 0.6877 - precision: 0.7564
Epoch 2/2
93/93 [==============================] - 2s 25ms/step - loss: 0.0289 - binary_accuracy: 0.9904 - recall: 0.7502 - precision: 0.8155
Loading images...
Preprocessing images...
Preprocessing TF...
Fitting model with 4443 lines...
Epoch 1/2
93/93 [==============================] - 2s 25ms/s